In [21]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np

url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees1= trees[['health','address']]
trees1.dropna(inplace=True)

app = dash.Dash()

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Question #1',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    html.Div(children='Proportion of trees in Good, Fair and Poor conditions', style={
        'textAlign': 'center',
        'color': colors['text']
    }),
    dcc.Graph(
        id='Graph1',
        figure={
            'data': [
                {'x': ['Good', 'Fair', 'Poor'], 'y': [trees1[trees1['health']=='Good'].shape[0]/trees1.shape[0]*100, trees1[trees1['health']=='Fair'].shape[0]/trees1.shape[0]*100, trees1[trees1['health']=='Poor'].shape[0]/trees1.shape[0]*100], 'type': 'bar'}
            ],
            'layout': {
                'plot_bgcolor': colors['background'],
                'paper_bgcolor': colors['background'],
                'font': {
                    'color': colors['text']
                }
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)